In [23]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import defaultdict
from cap_from_youtube import cap_from_youtube
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # to avoid OMP conficts

In [24]:
# Load an official or custom model
# model = YOLO('../models/yolov8n.pt')  # Load an official Detect model
model = YOLO('../models/yolov8x.pt')  # Load an official Detect model

# Tracking from a video file or YouTube url

In [4]:
# Open the video file
# fname="highway_drone_footage"
# fname="apollo15_10_1_1st"
# fname="cars"
# cap = cv2.VideoCapture("../videos/"+fname+".mp4")

# Open the video from YouTube
fname="from_yt_5"
# cap = cap_from_youtube("https://youtu.be/7HaJArMDKgI?si=T0Bb3zOcz-YiOnMF", '720p') # 1
# cap = cap_from_youtube("https://youtu.be/Y1jTEyb3wiI?si=-gU51avblW5Qo-ij", '720p') # 2
# cap = cap_from_youtube("https://youtu.be/jM2VrPE5kFg?si=RXfsB63fA58TuBSi", '720p') # 3
# cap = cap_from_youtube("https://youtu.be/CftLBPI1Ga4?si=N69rX1Bo3w9S6qjq", '1080p') # 4 (busy street with cars)
cap = cap_from_youtube("https://youtu.be/dDE3d6mEfn8?si=s5fAKjGVaHhfeLCQ", '2160p50') # 4 (mumbai street)


[youtube] Extracting URL: https://youtu.be/dDE3d6mEfn8?si=s5fAKjGVaHhfeLCQ
[youtube] dDE3d6mEfn8: Downloading webpage
[youtube] dDE3d6mEfn8: Downloading ios player API JSON
[youtube] dDE3d6mEfn8: Downloading player dee49cfa
[youtube] dDE3d6mEfn8: Downloading m3u8 information


In [5]:
# Store the track history
track_history = defaultdict(lambda: [])

# Below VideoWriter object will create a frame of above defined The output  
# is stored in 'output.mp4' file. 
output_video = cv2.VideoWriter("../output/"+fname+"_output.mp4",  
                         cv2.VideoWriter_fourcc(*'mp4v'), # writer object # XVID
                        #  cv2.VideoWriter_fourcc(*'XVID'),
                        #  cv2.VideoWriter_fourcc('m', 'p', '4', 'v'),
                         int(cap.get(cv2.CAP_PROP_FPS)), # FPS
                         (852,480)) # frame size

# Loop through the video frames
i=0
while cap.isOpened():
    i+=1
    # Read a frame from the video
    success, frame = cap.read()

    if success:

        # print(frame.shape)
        frame = cv2.resize(frame, dsize=(852,480))# reshape to 480, 852
        # print(frame.shape)

        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,tracker="bytetrack.yaml") # , show=True, stream=True

        try:
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(255,0,0), thickness=10)

        except:
            annotated_frame = frame

        # Display the annotated frame (requires X-forwarding ...)
        # cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # write the frame to the output file
        output_video.write(annotated_frame)

        # save every n-th frame as jpg
        n=50
        if i % n == 0:
            print('saving '+str(n)+'-th frame ...')
            cv2.imwrite("../output/"+fname+"_frame_%d.jpg" % i, annotated_frame)     # save frame as JPEG file 

        # except:
        #     print("No detection in this frame, skipping ...")



        if i==800:
            print("ok, that's enough ...")
            break

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print("failed to read the frame :<")
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
output_video.release()
cv2.destroyAllWindows()

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.0 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 1.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 (no detections), 63.9ms
Speed: 2.5ms preprocess, 63.9ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.5ms
Speed: 1.4ms preprocess, 28.5ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.4ms
Speed: 1.0ms preprocess, 28.4ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.3ms
Speed: 1.2ms preprocess, 13.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 31.8ms
Speed: 1.1ms preprocess, 31.8ms inference, 82.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

# Tracking on KITTI

In [25]:
from ultralytics import YOLO

model = YOLO("../models/yolov8n.pt")

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../utils/") # Adds utils python modules path.
from kitti_foundation import Kitti, Kitti_util
%matplotlib inline

v2c_filepath = '../datasets/KITTI/2011_09_26/calib_velo_to_cam.txt'
c2c_filepath = '../datasets/KITTI/2011_09_26/calib_cam_to_cam.txt'
xml_path = "../datasets/KITTI/2011_09_26/2011_09_26_drive_0005_sync/tracklet_labels.xml"

velo_path = '../datasets/KITTI/2011_09_26/2011_09_26_drive_0005_sync/velodyne_points/data'
camera_path = '../datasets/KITTI/2011_09_26/2011_09_26_drive_0005_sync/'

frame_no = 115
v_fov, h_fov = (-24.9, 2.0), (-90, 90) # field of view

In [27]:
def draw_tracklets(check):
    """ draw 3d bounding boxes around annotated objects """
    import cv2
    tracklet_, type_ = check.tracklet_info

    tracklet2d = []
    for i, j in zip(tracklet_[frame_no], type_[frame_no]):
        point = i.T
        chk,_ = check._Kitti_util__velo_2_img_projection(point)
        tracklet2d.append(chk)

    type_c = { 'Car': (0, 0, 255), 'Van': (0, 255, 0), 'Truck': (255, 0, 0), 'Pedestrian': (0,255,255), \
        'Person (sitting)': (255, 0, 255), 'Cyclist': (255, 255, 0), 'Tram': (0, 0, 0), 'Misc': (255, 255, 255)}

    line_order = ([0, 1], [1, 2],[2, 3],[3, 0], [4, 5], [5, 6], \
            [6 ,7], [7, 4], [4, 0], [5, 1], [6 ,2], [7, 3])

    for i, j in zip(tracklet2d, type_[frame_no]):
        for k in line_order:    
            cv2.line(image, (int(i[0][k[0]]), int(i[1][k[0]])), (int(i[0][k[1]]), int(i[1][k[1]])), type_c[j], 2)
        cv2.putText(image, j, (int(i[0][k[0]]), int(-10+i[1][k[0]])), cv2.FONT_HERSHEY_PLAIN, 1, type_c[j], 2)

Just using a regular camera image

In [18]:
# read left camera image
with_tracklets = True
image_type = 'color'  # 'gray' or 'color' image
mode = '00' if image_type == 'gray' else '02'  # image_00 = 'gray image' , image_02 = 'color image'

image_path = camera_path+'image_' + mode + '/data'
check = Kitti_util(frame=frame_no, velo_path=velo_path, camera_path=image_path, \
                   xml_path=xml_path, v2c_path=v2c_filepath, c2c_path=c2c_filepath)
image = check.camera_file

results = model.track(source=image, persist=True,tracker="bytetrack.yaml")

txt=""
if with_tracklets:
    draw_tracklets(check)
    txt="_with_tracklets"

for result in results:
    result.save(filename="../output/result_only_camera_frame_"+str(frame_no)+""+txt+".jpg")  # save to disk


parsing tracklet file ../datasets/KITTI/2011_09_26/2011_09_26_drive_0005_sync/tracklet_labels.xml
file contains 15 tracklets
loaded 15 tracklets

0: 224x640 4 cars, 4.3ms
Speed: 3.6ms preprocess, 4.3ms inference, 11.9ms postprocess per image at shape (1, 3, 224, 640)


In [ ]:
# results "cheat sheet":
# # Process results list
# for result in results:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     obb = result.obb  # Oriented boxes object for OBB outputs
#     result.show()  # display to screen
#     result.save(filename="result.jpg")  # save to disk

Using a camera image with projected lidar data

In [32]:
def print_projection_plt(points, color, image):
    """ project converted velodyne points into camera image """
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    for i in range(points.shape[1]):
        cv2.circle(hsv_image, (np.int32(points[0][i]),np.int32(points[1][i])),1, (int(color[i]),255,255),-1)

    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)

In [33]:
image_type = 'color'  # 'gray' or 'color' image
mode = '00' if image_type == 'gray' else '02'  # image_00 = 'gray image' , image_02 = 'color image'

image_path = camera_path+'image_' + mode + '/data'
res = Kitti_util(frame=frame_no, camera_path=image_path, velo_path=velo_path, \
                v2c_path=v2c_filepath, c2c_path=c2c_filepath)

img, pnt, c_ = res.velo_projection_frame(v_fov=v_fov, h_fov=h_fov)

image = print_projection_plt(pnt, c_, img)

results = model.track(source=image, persist=True,tracker="bytetrack.yaml")

txt=""

for result in results:
    result.save(filename="../output/result_camera_with_lidar_proj_frame_"+str(frame_no)+""+txt+".jpg")  # save to disk



0: 224x640 1 person, 18.5ms
Speed: 4.7ms preprocess, 18.5ms inference, 19.7ms postprocess per image at shape (1, 3, 224, 640)
